<a href="https://colab.research.google.com/github/tktmyd/ipynbs/blob/main/fdm/FDM4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2次元SH問題

In [ ]:
import numpy as np
import sys
if 'google.colab' in sys.modules:
    print("Installing pygmt on Google Colab. It may take a few minutes.")
    ! pip install -q condacolab &> /dev/null
    import condacolab
    condacolab.install()
    ! mamba install pygmt &> /dev/null
import pygmt

ここでは，発展問題として，2次元SH問題のスタガードグリッド差分法数値シミュレーションを行います．3次元の運動方程式と構成関係式に対して，$x_2$ 成分の微分がゼロであるという仮定のもと，$x_1$-$x_3$断面における波動伝播を考えます．すると，運動方程式と構成関係式は2組の方程式に分離することができ，そのうちの片方は以下のようなものになります．
$$
\begin{align}
		 & \rho \frac{\partial v_2}{\partial t} =
		\frac{\partial \sigma_{13}}{\partial x} +
		\frac{\partial \sigma_{23}}{\partial z}
		\\
		 & \frac{\partial \sigma_{23}}{\partial t} =
		\mu
		\frac{\partial v_2}{\partial x_3}
    , \quad \frac{\partial \sigma_{12}}{\partial t} =
		\mu
		\frac{\partial v_2}{\partial x_1}
\end{align}
$$

もし$x_3$方向を深さとして取ると，$v_2$は面外方向に水平に振動する成分ですから，$SH$波を表します．ここに示していない残りの$v_1$, $v_3$に関する方程式は $P$-$SV$ 波を表します．

表記を簡単にするため，$x_1\rightarrow x$, $x_3 \rightarrow z$と置き換えます．すると上記の方程式は

$$
\begin{align}
		 & \frac{\partial v_y}{\partial t} =
		 \frac{1}{\rho} \left(
		\frac{\partial \sigma_{xy}}{\partial x} +
		\frac{\partial \sigma_{yz}}{\partial z}
		\right)
		\\
		 & \frac{\partial \sigma_{yz}}{\partial t} =
		\mu
		\frac{\partial v_y}{\partial z}
		%\\
		% & 
    , \quad \frac{\partial \sigma_{xy}}{\partial t} =
		\mu
		\frac{\partial v_y}{\partial x}
\end{align}
$$

と表されます．もし$\mu$が空間一様ならば，運動方程式の時間微分に構成関係式を代入すると，

$$
\frac{\partial^2 v_y}{\partial t^2} = \frac{\mu}{\rho}
\left( \frac{\partial^2 v_y}{\partial x^2} + \frac{\partial^2 v_y}{\partial z^2} \right)
$$
となり，これは速度$\beta = \sqrt{\mu/\rho}$ の2次元波動方程式にほかなりません．

## 方程式の離散化

1次元の問題と同じように，まず空間微分から離散化しましょう．

$$
x_I = x_0 + I \Delta x, z_K = z_0 + K \Delta z
$$

というふうに，$x, z$方向をそれぞれ$\Delta x$, $\Delta z$で離散化します．そのときの値を，たとえば

$$
v_{y\,(I-1/2,K-1/2)}(t) = v_y (x_{I-1/2}, z_{K-1/2}, t)
$$
のように表します．スタガードグリッドでは必要に応じてグリッド位置が 1/2 だけズレるのも，1次元の問題と同じです．

うまく位置を組み合わせると，$SH$問題の方程式は，

$$
\begin{align}
& \sigma_{xy\,(I, K-1/2)}(t+\Delta t/2)
=
\sigma_{xy\,(I, K-1/2)}(t-\Delta t/2) +
\mu_{(I, K-1/2)}
\frac{v_{y,(I+1/2,K-1/2)} (t) - v_{y,(I-1/2,K-1/2)}(t)}{\Delta x} \Delta t
\\
 & \sigma_{yz\,(I-1/2, K)}(t+\Delta t/2)
= \sigma_{yz\,(I-1/2, K)}(t-\Delta t/2) +
	\mu_{(I-1/2, K)}
    \frac{v_{y,(I-1/2,K+1/2)}(t) - v_{y,(I-1/2,K-1/2)}(t)}{\Delta z} \Delta t
\\
& 		v_{y\,(I-1/2, K-1/2)}(t+\Delta t)
		=
		v_{y\,(I-1/2, K-1/2)}(t)
		\notag \\&\quad +
		\frac{1}{\rho_{(I-1/2, K-1/2)}} \left[
            \frac{\sigma_{xy\,(I, K-1/2)}(t+\Delta t/2) - \sigma_{xy\,(I-1, K-1/2)}(t+\Delta t/2)}{\Delta x} 
            \right. \notag\\
            &\qquad \left. 
            +
            \frac{\sigma_{yz\,(I-1/2, K)}(t+\Delta t/2) - \sigma_{yz\,(I-1/2, K-1)}(t+\Delta t/2)}{\Delta z}
        \right] \Delta t
\end{align}
$$